Vamos a entrenar una CNN, esta a ver a Color y con CIFAR-10.

In [4]:
# Arranque: imports, dataset y primer batch (MNIST)
import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


torch.manual_seed(3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Aqui convertimos las imagenes a tensores
# Es decir, el valor de los pixeles pasa a estar entre 0 y 1
# y el shape de las imagenes pasa a ser (C,H,W) (Canales, Alto, Ancho)

transform = transforms.ToTensor()  # [0,1], shape (C,H,W)




train = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

loader_train = DataLoader(train, batch_size=128, shuffle=True)
loader_test = DataLoader(test, batch_size=128, shuffle=True)


images, labels = next(iter(loader_train))
print(images.shape, labels.shape)





cuda
torch.Size([128, 3, 32, 32]) torch.Size([128])


Vamos a Normalizar los canales, esto es una buena practica y es común hacerlo.

Se cargan todas las 50 000 imágenes de entrenamiento.

Cada imagen se convierte a valores entre 0 y 1. (lo hicimos previamente con ToTensor)

Ahora:

Para cada canal (R, G, B) se calcula:

media → el promedio de todos los valores de ese canal en todas las imágenes;

desviación típica → cuánto varían esos valores respecto a la media.

In [ ]:
#Paso 1: Convertir a float64 para que los calculos sean exactos


#En PyTorch, cada tensor tiene dimensiones (también llamadas axes o dims).
#Por ejemplo, un tensor de 3 dimensiones (3D) tiene forma (B, C, H, W)
#Donde:
#B = Batch size (numero de imagenes en el batch)
#C = Canales (RGB, 3)
#H = Alto
#W = Ancho

# 2. Inicializamos acumuladores
#Son tensores 1D con tantos elementos como canales (R, G, B)
# Seria algo como [0,0,0] con una precision de 64 bits para sus decimales
sum_c = torch.zeros(3, dtype=torch.float64)
sum_sq_c = torch.zeros(3, dtype=torch.float64)
num_pixels = 0

# 3. Bucle sobre cada batch
for batch, _ in loader_train:
    #Convertirmos  ese batch a float64 tambien
    x = batch.to(torch.float64)
    #Aseguramos que el shape sea (128, 3, 32, 32)
    b, c, h, w = x.shape

    # Sumaremos todos los valores de todos los píxeles de todas las imágenes,
    # pero mantiene separados los tres canales (R, G, B).
    #
    # dim=(0,2,3) significa:
    #  - dim=0 → recorre y suma todas las imágenes del batch
    #  - dim=2 → recorre y suma todas las filas de píxeles (alto)
    #  - dim=3 → recorre y suma todas las columnas de píxeles (ancho)
    #
    # No se indica dim=1 (los canales) precisamente para NO sumarlos,
    # así PyTorch conserva esa dimensión y devuelve un vector con 3 valores:
    #   sum_c[0] → suma total de todos los píxeles del canal rojo (R)
    #   sum_c[1] → suma total de todos los píxeles del canal verde (G)
    #   sum_c[2] → suma total de todos los píxeles del canal azul (B)
    sum_c += x.sum(dim=(0, 2, 3))


    # Suma de cuadrados por canal
    sum_sq_c += (x ** 2).sum(dim=(0, 2, 3))
    # Contamos cuántos píxeles llevamos acumulados
    num_pixels += b * h * w

# 4. Cálculo de medias y desviaciones
mean = sum_c / num_pixels
std = ((sum_sq_c / num_pixels) - mean**2).sqrt()

print("mean:", mean)
print("std :", std)












In [ ]:
A